In [0]:
%sql
-- Creating the Bronze table structure
CREATE TABLE IF NOT EXISTS main.db_project.bronze_transactions (
  cost DOUBLE,
  currency STRING,
  marka STRING,
  model STRING,
  year INT,
  has_license STRING,
  place STRING,
  date STRING,
  id STRING,
  engine STRING,
  load_dt TIMESTAMP, -- Audit: When it was loaded
  source STRING      -- Audit: Where it came from
)
USING DELTA;

In [0]:
%sql
COPY INTO main.db_project.bronze_transactions
FROM (
  SELECT 
    cast(cost AS DOUBLE),
    cast(currency AS STRING),
    cast(marka AS STRING),
    cast(model AS STRING),
    try_cast(year AS INT) AS year, -- Will turn 'год' into NULL instead of failing
    cast(has_license AS STRING),
    cast(place AS STRING),
    cast(date AS STRING),
    cast(id AS STRING),
    cast(engine AS STRING),
    current_timestamp() AS load_dt, 
    _metadata.file_path AS source 
  FROM '/Volumes/main/db_project/raw_data/chunks/chunk1_initial'
)
FILEFORMAT = CSV
FORMAT_OPTIONS ('header' = 'true', 'inferSchema' = 'true')
COPY_OPTIONS ('mergeSchema' = 'true');

In [0]:
%sql
COMMENT ON TABLE main.db_project.bronze_transactions 
IS 'Bronze layer table for raw automobile transaction data ingested from CSV.';

ALTER TABLE main.db_project.bronze_transactions 
CHANGE COLUMN load_dt COMMENT 'Timestamp indicating when the record was ingested into the Bronze layer';